# Visualisation

Cleaner Version only including correct CFs

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

THR = 0.04  # threshold for highlighting CF changes. Needed sincse autoencoded methods can have negligible changes across many indices

def _collect_glacier(root_path, kind, meth):
    out = []
    for npz_file in (root_path / kind).glob(f"cf_fold*_{meth}.npz"):
        with np.load(npz_file) as Z:
            for i, (o, c, t, a, b) in enumerate(zip(Z["x_orig"], Z["x_cf"], Z["y_true"], Z["y_pred"], Z["y_cf"])):
                out.append({"x0": o, "xcf": c, "y_true": int(t), "y0": int(a), "ycf": int(b)})
    return out

def _collect_cels(root_path, mode):
    out = []
    for npz_file in (root_path / mode).glob("cf_fold*.npz"):
        with np.load(npz_file, allow_pickle=True) as Z:
            for i in range(len(Z["x0"])):
                out.append({"x0": Z["x0"][i], "xcf": Z["xcf"][i],
                            "y_true": int(Z["y_true"][i]),
                            "y0": int(Z["y_pred"][i]),
                            "ycf": int(Z["y_cf"][i])})
    return out

def _collect_rsf(root_path, mode):
    out = []
    for npz_file in (root_path / mode / "rsf").glob("cf_fold*.npz"):
        with np.load(npz_file) as Z:
            for i in range(len(Z["x0"])):
                out.append({"x0": Z["x0"][i], "xcf": Z["xcf"][i],
                            "y_true": int(Z["y_true"][i]),
                            "y0": int(Z["y_pred"][i]),
                            "ycf": int(Z["y_cf"][i])})
    return out

def plot_cf_example(ax, item, title):
    x0, xcf = np.squeeze(item["x0"]), np.squeeze(item["xcf"])
    delta = np.abs(xcf - x0)
    changed = delta > THR

    ax.plot(x0, lw=0.7, color="tab:red", label="Original")
    ax.plot(xcf, lw=1.1, color="grey", ls=":", label="CF")
    ax.fill_between(np.arange(len(x0)), 0, 1, where=changed, color="gold", alpha=0.3)

    ax.set_title(title)
    ax.legend()

def view(dataset_name, glacier_methods=["cnn", "ne"], meaningful_only=False):
    GLACIER_ROOT = Path("Glacier/learning-time-series-counterfactuals/cf_runs") / dataset_name
    CELS_ROOT = Path("CELS/CELS/cf_runs") / dataset_name
    RSF_ROOT = Path("RSF/RSF/cf_runs") / dataset_name

    # load data
    glacier_data = {f"{m}_local": _collect_glacier(GLACIER_ROOT, "local", m) for m in glacier_methods}
    glacier_data.update({f"{m}_global": _collect_glacier(GLACIER_ROOT, "global", m) for m in glacier_methods})
    cels_local, cels_global = _collect_cels(CELS_ROOT, "local"), _collect_cels(CELS_ROOT, "global")
    rsf_local, rsf_global = _collect_rsf(RSF_ROOT, "local"), _collect_rsf(RSF_ROOT, "global")

    # work out how many rows
    num_rows = len(glacier_methods) + 2
    n = max(len(d) for d in list(glacier_data.values()) + [cels_local, cels_global, rsf_local, rsf_global] if d)

    def _show(idx=0):
        fig, axes = plt.subplots(num_rows, 2, figsize=(16, 4 * num_rows), sharex=True)

        rows = []
        for m in glacier_methods:
            rows.append((f"Glacier {m.upper()} Global", glacier_data[f"{m}_global"],
                         f"Glacier {m.upper()} Local",  glacier_data[f"{m}_local"]))
        rows += [("CELS Global", cels_global, "CELS Local", cels_local),
                 ("RSF Global", rsf_global, "RSF Local", rsf_local)]

        for row, (gt, gdata, lt, ldata) in enumerate(rows):
            axg, axl = axes[row, 0], axes[row, 1]
            if gdata:
                plot_cf_example(axg, gdata[idx % len(gdata)], gt)
            else:
                axg.text(0.5, 0.5, "No data", ha="center", va="center")
            if ldata:
                plot_cf_example(axl, ldata[idx % len(ldata)], lt)
            else:
                axl.text(0.5, 0.5, "No data", ha="center", va="center")

        axes[-1, 0].set_xlabel("Spectral Index")
        axes[-1, 1].set_xlabel("Spectral Index")
        plt.tight_layout()
        plt.show()

    interact(_show, idx=IntSlider(0, 0, n - 1, 1, description="Sample Index"))

In [ ]:
view(dataset_name="EcoliVsKpneumoniae_ramanspy_singular", meaningful_only=True)
view(dataset_name="RamanCOVID19_ramanspy_preprocessed", meaningful_only=True)
view(dataset_name="DRS_TissueClassification", meaningful_only=True)

Loading Glacier CNN (Local)...
Loading Glacier CNN (Global)...
Loading Glacier NE (Local)...
Loading Glacier NE (Global)...
Loading CELS (Local)...
Loading CELS (Global)...
Loading RSF (Local)...
Loading RSF (Global)...

Filtered to 8 indices where ALL methods have meaningful CFs


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample Index', max=7), Output()…

Loading Glacier CNN (Local)...
Loading Glacier CNN (Global)...
Loading Glacier NE (Local)...
Loading Glacier NE (Global)...
Loading CELS (Local)...
Loading CELS (Global)...
Loading RSF (Local)...
Loading RSF (Global)...

Filtered to 58 indices where ALL methods have meaningful CFs


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample Index', max=57), Output(…

Loading Glacier CNN (Local)...
Loading Glacier CNN (Global)...
Loading Glacier NE (Local)...
Loading Glacier NE (Global)...
Loading CELS (Local)...
Loading CELS (Global)...
Loading RSF (Local)...
Loading RSF (Global)...

Filtered to 129 indices where ALL methods have meaningful CFs


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample Index', max=128), Output…

This version Gives all CFs and provides statistics

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

THR = 0.04

def _collect_glacier(root_path, kind, meth):
    out = []
    data_path = root_path / kind
    for npz_file in data_path.glob(f"cf_fold*_{meth}.npz"):
        with np.load(npz_file) as Z:
            for i, (o, c, t, a, b) in enumerate(zip(Z["x_orig"], Z["x_cf"], Z["y_true"], Z["y_pred"], Z["y_cf"])):
                out.append({
                    "x0": o, "xcf": c,
                    "y_true": int(t), "y0": int(a), "ycf": int(b),
                    "tag": f"{npz_file.name}:{i}"
                })
    if not out:
        print(f"Warning: No Glacier CFs found for {data_path}")
    return out

def _collect_cels(root_path, mode):
    out = []
    data_path = root_path / mode
    for npz_file in data_path.glob("cf_fold*.npz"):
        with np.load(npz_file, allow_pickle=True) as Z:
            for i in range(len(Z["x0"])):
                out.append({
                    "x0": Z["x0"][i], "xcf": Z["xcf"][i],
                    "y_true": int(Z["y_true"][i]),
                    "y0": int(Z["y_pred"][i]),
                    "ycf": int(Z["y_cf"][i]),
                    "tag": f"{npz_file.name}:{i} ({Z['tag'][i]})"
                })
    if not out:
        print(f"Warning: No CELS CFs found for {data_path}")
    return out

def _collect_rsf(root_path, mode):
    out = []
    data_path = root_path / mode / "rsf"
    for npz_file in data_path.glob("cf_fold*.npz"):
        with np.load(npz_file) as Z:
            for i in range(len(Z["x0"])):
                out.append({
                    "x0": Z["x0"][i], "xcf": Z["xcf"][i],
                    "y_true": int(Z["y_true"][i]),
                    "y0": int(Z["y_pred"][i]),
                    "ycf": int(Z["y_cf"][i]),
                    "tag": f"{npz_file.name}:{i}"
                })
    if not out:
        print(f"Warning: No RSF CFs found for {data_path}")
    return out

def view(dataset_name, glacier_meth="cnn"):
    GLACIER_ROOT = Path("Glacier/learning-time-series-counterfactuals/cf_runs") / dataset_name
    CELS_ROOT = Path("CELS/CELS/cf_runs") / dataset_name
    RSF_ROOT = Path("RSF/RSF/cf_runs") / dataset_name

    print("Loading Glacier (Local)...")
    glacier_local = _collect_glacier(GLACIER_ROOT, "local", glacier_meth)
    print("Loading Glacier (Global)...")
    glacier_global = _collect_glacier(GLACIER_ROOT, "global", glacier_meth)

    print("Loading CELS (Local)...")
    cels_local = _collect_cels(CELS_ROOT, "local")
    print("Loading CELS (Global)...")
    cels_global = _collect_cels(CELS_ROOT, "global")

    print("Loading RSF (Local)...")
    rsf_local = _collect_rsf(RSF_ROOT, "local")
    print("Loading RSF (Global)...")
    rsf_global = _collect_rsf(RSF_ROOT, "global")

    n = max(len(d) for d in [glacier_local, glacier_global, cels_local, cels_global, rsf_local, rsf_global] if d)

    def _plot_cf(ax, item, title):
        x0, xcf = np.squeeze(item["x0"]), np.squeeze(item["xcf"])
        y_true, y0, ycf, tag = item["y_true"], item["y0"], item["ycf"], item["tag"]

        delta = np.abs(xcf - x0)
        changed = delta > THR
        sparsity = np.mean(changed)
        proximity = np.linalg.norm(x0 - xcf)
        meaningful_cf = (y0 == y_true) and (y0 != ycf)

        ax.plot(x0, lw=1, color="#d62728", label=f"Original (true={y_true}, pred={y0})")
        ax.plot(xcf, lw=0.7, color="grey", ls="--", label=f"CF (ŷ={ycf})")
        ax.fill_between(np.arange(len(x0)), 0, 1, where=changed, color="#ffdf88", alpha=0.35)

        note = " [Meaningful CF]" if meaningful_cf else ""
        ax.set_title(
            f"{title} | Sparsity: {sparsity:.1%} | Proximity: {proximity:.2f}{note}\n(Source: {tag})",
            fontsize=10
        )
        ax.legend(loc="upper right")

    def _show(idx=0):
        fig, axes = plt.subplots(3, 2, figsize=(16, 12), sharex=True)
        layout = [
            ("Glacier GLOBAL", glacier_global, "Glacier LOCAL", glacier_local),
            ("CELS GLOBAL", cels_global, "CELS LOCAL", cels_local),
            ("RSF GLOBAL", rsf_global, "RSF LOCAL", rsf_local)
        ]

        for row, (gt, gdata, lt, ldata) in enumerate(layout):
            axg, axl = axes[row, 0], axes[row, 1]
            if gdata:
                _plot_cf(axg, gdata[idx % len(gdata)], gt)
            else:
                axg.set_title(f"{gt} - No data found", fontsize=10)
                axg.text(0.5, 0.5, "No data", ha="center", va="center", style="italic")
            if ldata:
                _plot_cf(axl, ldata[idx % len(ldata)], lt)
            else:
                axl.set_title(f"{lt} - No data found", fontsize=10)
                axl.text(0.5, 0.5, "No data", ha="center", va="center", style="italic")

        axes[2, 0].set_xlabel("Spectral Index")
        axes[2, 1].set_xlabel("Spectral Index")
        plt.tight_layout()
        plt.show()

    interact(_show, idx=IntSlider(0, 0, n - 1, 1, description="Sample Index", continuous_update=False))


In [ ]:
view(dataset_name="EcoliVsKpneumoniae_ramanspy_singular")
view(dataset_name="RamanCOVID19_ramanspy_preprocessed")
view(dataset_name="DRS_TissueClassification")

Loading Glacier CNN (Local)...
Loading Glacier CNN (Global)...
Loading Glacier NE (Local)...
Loading Glacier NE (Global)...
Loading CELS (Local)...
Loading CELS (Global)...
Loading RSF (Local)...
Loading RSF (Global)...


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample Index', max=499), Output…

Loading Glacier CNN (Local)...
Loading Glacier CNN (Global)...
Loading Glacier NE (Local)...
Loading Glacier NE (Global)...
Loading CELS (Local)...
Loading CELS (Global)...
Loading RSF (Local)...
Loading RSF (Global)...


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample Index', max=308), Output…

Loading Glacier CNN (Local)...
Loading Glacier CNN (Global)...
Loading Glacier NE (Local)...
Loading Glacier NE (Global)...
Loading CELS (Local)...
Loading CELS (Global)...
Loading RSF (Local)...
Loading RSF (Global)...


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample Index', max=499), Output…

# Metrics

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

def per_sample_metrics(x0, xcf, y0, ycf, thr):
    x0 = np.asarray(x0).ravel()
    xcf = np.asarray(xcf).ravel()
    delta = xcf - x0
    
    l2_abs = float(np.linalg.norm(delta))
    l2_rel = float(l2_abs / (np.linalg.norm(x0) + 1e-12))
    sparsity = float(np.mean(np.abs(delta) > thr))
    validity = int(int(ycf) != int(y0))
    
    return l2_abs, l2_rel, sparsity, validity

def calculate_and_display_metrics(dataset_name, threshold=0.025):
    print(f"--- Calculating Metrics for: {dataset_name} ---")
    
    GLACIER_ROOT = Path("Glacier/learning-time-series-counterfactuals/cf_runs") / dataset_name
    CELS_ROOT = Path("CELS/CELS/cf_runs") / dataset_name
    RSF_ROOT = Path("RSF/RSF/cf_runs") / dataset_name

    all_sets = {
        "Glacier_cnn_local": _collect_glacier(GLACIER_ROOT, "local", "cnn"),
        "Glacier_cnn_global": _collect_glacier(GLACIER_ROOT, "global", "cnn"),
        "Glacier_ne_local": _collect_glacier(GLACIER_ROOT, "local", "ne"),
        "Glacier_ne_global": _collect_glacier(GLACIER_ROOT, "global", "ne"),
        "CELS_local": _collect_cels(CELS_ROOT, "local"),
        "CELS_global": _collect_cels(CELS_ROOT, "global"),
        "RSF_local": _collect_rsf(RSF_ROOT, "local"),
        "RSF_global": _collect_rsf(RSF_ROOT, "global"),
    }

    rows = []
    for name, items in all_sets.items():
        if not items:
            print(f"No '{name}', skipping.")
            continue
            
        for it in items:
            l2_abs, l2_rel, spars, valid = per_sample_metrics(it["x0"], it["xcf"], it["y0"], it["ycf"], threshold)
            rows.append({
                "Method": name,
                "proximity_abs": l2_abs,
                "proximity_rel": l2_rel,
                "sparsity": spars,
                "validity": valid,
            })

    df = pd.DataFrame(rows)
    
    agg = df.groupby("Method").agg(
        N=("validity", "size"),
        **{
            "Avg Rel Proximity": ("proximity_rel", "mean"),
            "Std Rel Proximity": ("proximity_rel", "std"),
            "Median Rel Proximity": ("proximity_rel", "median"),
            "Avg Sparsity": ("sparsity", "mean"),
            "Std Sparsity": ("sparsity", "std"),
            "Median Sparsity": ("sparsity", "median"),
            "Validity": ("validity", "mean"),
        }
    ).reset_index()

    pretty = agg.copy()
    pretty["Avg Rel Proximity"] = pretty["Avg Rel Proximity"].round(4)
    pretty["Std Rel Proximity"] = pretty["Std Rel Proximity"].round(4)
    pretty["Median Rel Proximity"] = pretty["Median Rel Proximity"].round(4)
    pretty["Avg Sparsity"] = (pretty["Avg Sparsity"] * 100).round(2).astype(str) + "%"
    pretty["Std Sparsity"] = (pretty["Std Sparsity"] * 100).round(2).astype(str) + "%"
    pretty["Median Sparsity"] = (pretty["Median Sparsity"] * 100).round(2).astype(str) + "%"
    pretty["Validity"] = (pretty["Validity"] * 100).round(2).astype(str) + "%"

    print("\n--- Aggregated Results ---")
    display(pretty.style.hide(axis='index'))
    return agg

def compare_methods_statistical(dataset_name, threshold=0.025):
    """Additional function to provide statistical comparisons between methods"""
    print(f"--- Statistical Comparison for: {dataset_name} ---")

    # Get raw aggregated data
    agg = calculate_and_display_metrics(dataset_name, threshold)

    if agg is None:
        return
    # Find best performing methods for each metric
    print("\n--- Best Performers ---")
    best_validity = agg.loc[agg['Validity'].idxmax()]
    print(f"Best Validity: {best_validity['Method']} ({best_validity['Validity']:.1%})")

    best_proximity = agg.loc[agg['Avg Rel Proximity'].idxmin()]
    print(f"Best Proximity: {best_proximity['Method']} ({best_proximity['Avg Rel Proximity']:.4f})")

    best_sparsity = agg.loc[agg['Avg Sparsity'].idxmin()]
    print(f"Best Sparsity: {best_sparsity['Method']} ({best_sparsity['Avg Sparsity']:.1%})")

    #return agg

In [59]:
compare_methods_statistical("DRS_TissueClassification")
compare_methods_statistical("EcoliVsKpneumoniae_ramanspy_singular")
compare_methods_statistical("RamanCOVID19_ramanspy_preprocessed")

--- Statistical Comparison for: DRS_TissueClassification ---
--- Calculating Metrics for: DRS_TissueClassification ---

--- Aggregated Results ---


Method,N,Avg Rel Proximity,Std Rel Proximity,Median Rel Proximity,Avg Sparsity,Std Sparsity,Median Sparsity,Validity
CELS_global,500,0.074100,0.029000,0.070200,31.99%,10.2%,32.4%,99.2%
CELS_local,500,0.028200,0.016900,0.028100,3.34%,2.52%,2.78%,48.2%
Glacier_global,500,0.049600,0.016300,0.046000,16.97%,3.49%,16.92%,99.4%
Glacier_local,500,0.055600,0.018900,0.053600,24.49%,16.51%,18.06%,98.8%
RSF_global,1250,0.134300,0.048500,0.136700,9.82%,0.62%,9.99%,55.92%
RSF_local,1250,0.081900,0.032100,0.080300,45.92%,17.38%,47.39%,98.88%



--- Best Performers ---
Best Validity: Glacier_global (99.4%)
Best Proximity: CELS_local (0.0282)
Best Sparsity: CELS_local (3.3%)
--- Statistical Comparison for: EcoliVsKpneumoniae_ramanspy_singular ---
--- Calculating Metrics for: EcoliVsKpneumoniae_ramanspy_singular ---

--- Aggregated Results ---


Method,N,Avg Rel Proximity,Std Rel Proximity,Median Rel Proximity,Avg Sparsity,Std Sparsity,Median Sparsity,Validity
CELS_global,500,0.094900,0.013100,0.092900,18.95%,1.79%,18.8%,25.0%
CELS_local,500,0.140800,0.023500,0.143900,10.91%,4.59%,10.7%,76.4%
Glacier_global,500,0.076700,0.015100,0.074400,17.21%,3.67%,16.3%,98.8%
Glacier_local,500,0.069000,0.016600,0.066200,16.7%,5.97%,16.0%,98.8%
RSF_global,1250,0.101800,0.021400,0.098400,8.72%,0.54%,8.7%,7.52%
RSF_local,1250,0.072300,0.036600,0.078500,30.53%,17.38%,33.95%,82.72%



--- Best Performers ---
Best Validity: Glacier_global (98.8%)
Best Proximity: Glacier_local (0.0690)
Best Sparsity: RSF_global (8.7%)
--- Statistical Comparison for: RamanCOVID19_ramanspy_preprocessed ---
--- Calculating Metrics for: RamanCOVID19_ramanspy_preprocessed ---

--- Aggregated Results ---


Method,N,Avg Rel Proximity,Std Rel Proximity,Median Rel Proximity,Avg Sparsity,Std Sparsity,Median Sparsity,Validity
CELS_global,309,0.086200,0.046100,0.068700,8.61%,8.42%,5.22%,60.19%
CELS_local,349,0.048000,0.042300,0.032700,2.13%,2.61%,1.44%,69.91%
Glacier_global,309,0.358600,0.043000,0.352400,18.33%,1.27%,18.56%,100.0%
Glacier_local,309,0.398100,0.102900,0.438500,18.1%,7.28%,20.56%,100.0%
RSF_global,309,0.096900,0.043600,0.083300,5.52%,2.22%,5.56%,72.49%
RSF_local,309,0.110600,0.050600,0.113200,12.86%,10.16%,11.67%,89.32%



--- Best Performers ---
Best Validity: Glacier_global (100.0%)
Best Proximity: CELS_local (0.0480)
Best Sparsity: CELS_local (2.1%)


# Multiclass

### Loaders

In [47]:
def _load_glacier_concat(project_path: str, dataset_name: str, mode: str, method: str):
    base = Path(project_path) / "cf_runs" / dataset_name / mode
    files = sorted(base.glob(f"cf_fold*_{method}.npz"))
    if not files:
        print(f"  [Glacier] No files for {mode}/{method} under {base}")
        return None, None, None, None, None

    X0, XCF, YTRUE, YPRED, YCF = [], [], [], [], []
    for f in files:
        try:
            with np.load(f, allow_pickle=True) as Z:
                x0 = Z["x_orig"]
                xcf = Z["x_cf"]
                y_true = Z["y_true"]
                y_pred = Z["y_pred"]
                y_cf = Z["y_cf"]
                
            n = min(len(x0), len(xcf), len(y_true), len(y_pred), len(y_cf))
            X0.append(np.squeeze(x0[:n]))
            XCF.append(np.squeeze(xcf[:n]))
            YTRUE.append(y_true[:n])
            YPRED.append(y_pred[:n])
            YCF.append(y_cf[:n])
            print(f"  [Glacier] loaded {f.name}: {n} samples")
        except Exception as e:
            print(f"  [Glacier] WARNING: could not load {f}: {e}")

    x_orig = np.concatenate(X0, axis=0)
    x_cf = np.concatenate(XCF, axis=0)
    y_true = np.concatenate(YTRUE, axis=0)
    y_pred = np.concatenate(YPRED, axis=0)
    y_cf = np.concatenate(YCF, axis=0)
    print(f"  [Glacier] concatenated {mode}/{method}: {len(x_orig)} samples total")
    return x_orig, x_cf, y_true, y_pred, y_cf


def _load_cels_concat(project_path: str, dataset_name: str, mode: str):
    base = Path(project_path) / "cf_runs" / dataset_name / mode
    files = sorted(base.glob("cf_fold*.npz"))
    
    if not files:
        print(f"  [CELS] No files for {mode} under {base}")
        return None, None, None, None, None

    X0, XCF, YTRUE, YPRED, YCF = [], [], [], [], []
    for f in files:
        try:
            with np.load(f, allow_pickle=True) as Z:
                x0 = Z["x0"]
                xcf = Z["xcf"]
                y_true = Z["y_true"]
                y_pred = Z["y_pred"]
                y_cf = Z["y_cf"]
            
            n = min(len(x0), len(xcf), len(y_true), len(y_pred), len(y_cf))
            if n > 0:
                X0.append(np.squeeze(x0[:n]))
                XCF.append(np.squeeze(xcf[:n]))
                YTRUE.append(y_true[:n])
                YPRED.append(y_pred[:n])
                YCF.append(y_cf[:n])
                print(f"  [CELS] loaded {f.name}: {n} samples")
        except Exception as e:
            print(f"  [CELS] WARNING: could not load {f}: {e}")
    
    if not X0:
        return None, None, None, None, None
        
    try:
        x_orig = np.concatenate(X0, axis=0)
        x_cf = np.concatenate(XCF, axis=0)
        y_true = np.concatenate(YTRUE, axis=0)
        y_pred = np.concatenate(YPRED, axis=0)
        y_cf = np.concatenate(YCF, axis=0)
        print(f"  [CELS] concatenated {mode}: {len(x_orig)} samples total")
        return x_orig, x_cf, y_true, y_pred, y_cf
    except Exception as e:
        print(f"  [CELS] ERROR concatenating data: {e}")
        return None, None, None, None, None
    

def _load_rsf_concat(project_path: str, dataset_name: str, mode: str):
    base = Path(project_path) / "cf_runs" / dataset_name / mode / "rsf"
    
    if not base.exists():
        print(f"  [RSF] Directory not found: {base}")
        return None, None, None, None, None
        
    files = sorted(base.glob("cf_fold*.npz"))
    if not files:
        print(f"  [RSF] No files for {mode} under {base}")
        return None, None, None, None, None

    X0, XCF, YTRUE, YPRED, YCF = [], [], [], [], []
    for f in files:
        try:
            with np.load(f, allow_pickle=False) as Z:
                x0 = Z["x0"]
                xcf = Z["xcf"]
                y_true = Z["y_true"]
                y_pred = Z["y_pred"]
                y_cf = Z["y_cf"]
                
            n = min(len(x0), len(xcf), len(y_true), len(y_pred), len(y_cf))
            if n > 0:  
                X0.append(np.squeeze(x0[:n]))
                XCF.append(np.squeeze(xcf[:n]))
                YTRUE.append(y_true[:n])
                YPRED.append(y_pred[:n])
                YCF.append(y_cf[:n])
                print(f"  [RSF] loaded {f.name}: {n} samples")
        except Exception as e:
            print(f"  [RSF] WARNING: could not load {f}: {e}")

    if not X0:
        return None, None, None, None, None

    try:
        x_orig = np.concatenate(X0, axis=0)
        x_cf = np.concatenate(XCF, axis=0)
        y_true = np.concatenate(YTRUE, axis=0)
        y_pred = np.concatenate(YPRED, axis=0)
        y_cf = np.concatenate(YCF, axis=0)
        print(f"  [RSF] concatenated {mode}: {len(x_orig)} samples total")
        return x_orig, x_cf, y_true, y_pred, y_cf
    except Exception as e:
        print(f"  [RSF] ERROR concatenating data: {e}")
        return None, None, None, None, None

### Evaluation

In [ ]:
import ramanspy
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from IPython.display import display
from sklearn.svm import SVC
import time

def run_multiclass_evaluation(dataset_name, use_ramanspy_preprocessing=False, 
                             perform_gridsearch=True, quick_search=False):
    print("="*60)
    print(f"Running Multi-Class Evaluation for: {dataset_name}")
    if use_ramanspy_preprocessing:
        print("Preprocessing Mode: ramanspy")
    if perform_gridsearch:
        print("Hyperparameter Tuning: GridSearchCV enabled")
    print("="*60)

    # Configd
    DATASET_CONFIGS = {
        "EcoliVsKpneumoniae_ramanspy_singular": {
            "classes_to_use": [0, 1, 2, 3, 9], #'C. albicans', 'C. glabrata', 'K. aerogenes'
            "bin2orig": {1: {3}, 0: {9}}, #E.coli:3,4 K.pneumoniae:9,10
            "class_names": {0: "C.albicans", 1: "C.glabrata", 2: "K.aerogenes", 
                           3: "E.coli_1", 4: "E.coli_2", 9: "K.pneum_1", 10: "K.pneum_2"}
        },

        "RamanCOVID19_ramanspy_preprocessed": {
        "classes_to_use": [0, 1, 2],
        "bin2orig": {1: {1}, 0: {0}},
        "class_names": {0: "Healthy", 1: "COVID-19", 2: "Suspected"}
        },

        "DRS_TissueClassification": {
            "classes_to_use": [0, 1, 2, 3, 4, 5],  # All 6 tissue classes
            "bin2orig": {
                1: {1},  # cortBone
                0: {0}  # muscle
            },
            "class_names": {0: "muscle", 1: "cortBone", 2: "traBone", 3: "cartilage", 4: "boneMarrow", 5: "boneCement"}
        },
    }
    
    if dataset_name not in DATASET_CONFIGS:
        print(f"ERROR: No configuration found for dataset '{dataset_name}'. Please add it to DATASET_CONFIGS.")
        return

    config = DATASET_CONFIGS[dataset_name]
    BIN2ORIG = config["bin2orig"]
    CLASS_NAMES = config.get("class_names", {})
    OUT_CSV = Path(f"multiclass_metrics_{dataset_name}.csv")

    # Load reference data
    print("Loading reference spectra...")
    RAMAN_DIR = Path.home() / "data" / "raman"
    
    if dataset_name in ["RamanCOVID19", "RamanCOVID19_ramanspy_preprocessed"]:
        if dataset_name == "RamanCOVID19":
            csv_path = Path("/home/cok7/local-datasets/covid19/covid19_serum_raman.csv")
        else: 
            csv_path = Path("/home/cok7/local-datasets/covid19/covid19_serum_raman_preprocessed.csv")
        
        df = pd.read_csv(csv_path)
        y_names = df["diagnostic"].astype(str)
        name2id = {"Healthy": 0, "COVID-19": 1, "Suspected": 2}
        y_all = y_names.map(name2id).to_numpy(np.int64)
        X_all = df.drop(columns=["diagnostic"]).to_numpy(np.float32)

    elif dataset_name == "DRS_TissueClassification":
        csv_path = Path("/home/cok7/local-datasets/drs_tissue.csv")
        df = pd.read_csv(csv_path)
        y_names = df["tissue_type"].astype(str)
        
        # Map all tissue types to numeric classes
        name2id = {
            "muscle": 0, 
            "cortBone": 1, 
            "traBone": 2, 
            "cartilage": 3, 
            "boneMarrow": 4, 
            "boneCement": 5
        }
        
        y_all = df["tissue_type"].map(name2id).to_numpy(np.int64)
        X_all = df.drop(columns=["tissue_type"]).to_numpy(np.float32)
        print(f"DRS tissue distribution: {dict(zip(*np.unique(y_all, return_counts=True)))}")
        
    else:
        X_all = np.load(RAMAN_DIR / "X_reference.npy")
        y_all = np.load(RAMAN_DIR / "y_reference.npy")

    print(f"Loaded reference data: X_all.shape={X_all.shape}, y_all.shape={y_all.shape}")
    print(f"Unique classes in reference: {np.unique(y_all)}")

    # Apply preprocessing
    if use_ramanspy_preprocessing:
        print("Applying ramanspy preprocessing to reference data...")
        try:
            raman_spectra = ramanspy.Spectrum(X_all, np.arange(X_all.shape[1]))
            
            pipeline = ramanspy.preprocessing.Pipeline([
                ramanspy.preprocessing.despike.WhitakerHayes(),
                ramanspy.preprocessing.baseline.ASLS(),
                ramanspy.preprocessing.normalise.MinMax(),
            ])
            
            preprocessed_spectra = pipeline.apply(raman_spectra)
            X_all = preprocessed_spectra.spectral_data 
            print("Preprocessing completed successfully")
        except Exception as e:
            print(f"WARNING: Preprocessing failed: {e}")
            print("Continuing with unprocessed data...")

    # Filter to relevant classes
    mask = np.isin(y_all, config["classes_to_use"])
    X_sub, y_sub = X_all[mask].astype(np.float32), y_all[mask]
    
    print(f"After filtering to relevant classes: X_sub.shape={X_sub.shape}")
    print(f"Class distribution: {dict(zip(*np.unique(y_sub, return_counts=True)))}")

    if len(X_sub) == 0:
        print("ERROR: No samples found for the specified classes!")
        return None

    # Train/test split
    X_tr, X_te, y_tr, y_te = train_test_split(
        X_sub, y_sub, test_size=0.20, stratify=y_sub, random_state=4
    )

    # Train SVM model
    print(f"\n--- Training SVM ---")
    
    if perform_gridsearch:
        print(f"Performing hyperparameter tuning for SVM...")
        
        if quick_search:
            param_grid = {
                'C': [1, 10],
                'gamma': ['scale'],
                'kernel': ['rbf', 'linear']
            }
        else:
            param_grid = {
                'C': [0.1, 1, 10],
                'gamma': ['scale'],
                'kernel': ['rbf', 'linear', 'poly']
            }
        
        cv_folds = 3 if quick_search else 5
        cv_strategy = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=4)
        
        start_time = time.time()
        grid_search = GridSearchCV(
            SVC(random_state=4, probability=True), param_grid, cv=cv_strategy, 
            scoring='accuracy', n_jobs=-1, verbose=0, return_train_score=True
        )
        
        grid_search.fit(X_tr, y_tr)
        search_time = time.time() - start_time
        
        print(f"SVM grid search completed in {search_time:.1f} seconds")
        print(f"Best CV score: {grid_search.best_score_:.4f}")
        print(f"Best parameters: {grid_search.best_params_}")
        
        svm = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_cv_score = grid_search.best_score_
    else:
        print(f"Using default SVM parameters...")
        svm = SVC(random_state=4, probability=True)
        svm.fit(X_tr, y_tr)
        best_params = None
        best_cv_score = None

    # Evaluate on test set
    y_pred = svm.predict(X_te)
    val_acc = accuracy_score(y_te, y_pred)
    print(f"\n{len(config['classes_to_use'])}-class SVM test accuracy: {val_acc:.4f}")
    
    cm = confusion_matrix(y_te, y_pred, labels=config["classes_to_use"])
    print(f"Confusion Matrix:\n{cm}")
    
    target_names = [CLASS_NAMES.get(cls, f"Class_{cls}") for cls in config["classes_to_use"]]
    print(f"\nClassification Report:")
    print(classification_report(y_te, y_pred, labels=config["classes_to_use"], 
                            target_names=target_names, digits=3))

    # Continue with counterfactual evaluation...
    print("\nLoading and evaluating counterfactuals...")
    experiments = [
        ("Glacier", "Glacier/learning-time-series-counterfactuals", "local", "cnn"),
        ("Glacier", "Glacier/learning-time-series-counterfactuals", "global", "cnn"),
        ("Glacier", "Glacier/learning-time-series-counterfactuals", "local", "ne"),
        ("Glacier", "Glacier/learning-time-series-counterfactuals", "global", "ne"),
        ("CELS", "CELS/CELS", "local", "cels"),
        ("CELS", "CELS/CELS", "global", "cels"),
        ("RSF", "RSF/RSF", "local", "rsf"),
        ("RSF", "RSF/RSF", "global", "rsf"),
    ]

    results = []
    method_stats = defaultdict(lambda: {"total": 0, "strict_success": 0, "any_flip": 0, 
                                       "Baseline_Correct": 0, "invalid_cf": 0,
                                       "validated_correct": 0, "validated_strict_success": 0})
    
    for project_name, project_path, mode, method in experiments:
        full_name = f"{project_name}_{mode}_{method}"
        print(f"- Evaluating '{full_name}'")
        
        # Load counterfactuals
        try:
            if project_name == "Glacier":
                result = _load_glacier_concat(project_path, dataset_name, mode, method)
            elif project_name == "CELS":
                result = _load_cels_concat(project_path, dataset_name, mode)
            elif project_name == "RSF":
                result = _load_rsf_concat(project_path, dataset_name, mode)
            else:
                print(f"  WARNING: Unknown project name '{project_name}', skipping...")
                continue
            
            # Check if loading was successful
            if result is None or all(x is None for x in result):
                print(f"  Skipping '{full_name}': no CF files found.")
                continue
                
            x_orig, x_cf, y_true, y_pred, y_cf = result
            
            if x_orig is None:
                print(f"  Skipping '{full_name}': no valid data loaded.")
                continue
                
        except Exception as e:
            print(f"  ERROR loading '{full_name}': {e}")
            continue

        print(f"  Loaded {len(x_orig)} samples (after concat)")

        
        # Apply same preprocessing to CFs if needed
        should_preprocess_cfs = use_ramanspy_preprocessing and "ramanspy" not in dataset_name.lower()
        
        if should_preprocess_cfs:
            print(f"  Applying ramanspy preprocessing to CFs from {full_name}")
            try:
                orig_spectra = ramanspy.Spectrum(x_orig, np.arange(x_orig.shape[1]))
                cf_spectra = ramanspy.Spectrum(x_cf, np.arange(x_cf.shape[1]))
                
                x_orig = pipeline.apply(orig_spectra).spectral_data
                x_cf = pipeline.apply(cf_spectra).spectral_data
            except Exception as e:
                print(f"  WARNING: CF preprocessing failed: {e}")
        elif "ramanspy" in dataset_name.lower():
            print(f"  Skipping CF preprocessing - {dataset_name} CFs already preprocessed")

        svm_before = svm.predict(x_orig)
        svm_after = svm.predict(x_cf)
        
        stats = method_stats[full_name]
        
        for i in range(len(x_cf)):
            y_bin_orig = int(y_pred[i]) if np.isscalar(y_pred[i]) else int(y_pred[i][0])
            y_bin_cf = int(y_cf[i]) if np.isscalar(y_cf[i]) else int(y_cf[i][0])

                
                
            orig_family = BIN2ORIG[y_bin_orig]
            target_family = BIN2ORIG[y_bin_cf]

            svm_before_lbl = int(svm_before[i])
            svm_after_lbl = int(svm_after[i])

            # did classifier place the original in its own family?
            svm_before_ok = (svm_before_lbl in orig_family)
            
            # Did the ground truth match the original family?
            y_true_lbl = int(y_true[i]) if np.isscalar(y_true[i]) else int(y_true[i][0])
            ground_truth_ok = (y_true_lbl == y_bin_orig)

            # Any Flip
            any_ok = (svm_after_lbl not in orig_family)

            # Strict Flip. landed specifically in the intended target family
            strict_ok = any_ok and (svm_after_lbl in target_family)

            # was the CF different
            cf_different = not np.allclose(x_orig[i], x_cf[i], rtol=1e-3)

            # meaningful CF metric
            meaningful_cf = (svm_before_lbl in orig_family) and (svm_after_lbl not in orig_family)
            
            # Both ground truth AND SVM agree original was correct
            validated_correct = ground_truth_ok and svm_before_ok
            
            # Strict success from a validated correct original
            validated_strict_success = validated_correct and strict_ok

            # Distance metrics
            dist_abs = float(np.linalg.norm(x_orig[i] - x_cf[i]))
            denom = max(1e-8, np.linalg.norm(x_orig[i]))
            dist_rel = float(dist_abs / denom)

            stats["total"] += 1
            if svm_before_ok: stats["Baseline_Correct"] += 1
            if strict_ok: stats["strict_success"] += 1
            if any_ok: stats["any_flip"] += 1
            if not cf_different: stats["invalid_cf"] += 1
            if validated_correct: stats["validated_correct"] += 1
            if validated_strict_success: stats["validated_strict_success"] += 1

            results.append({
                "Method": full_name,
                "svm_before": svm_before_lbl,
                "svm_after": svm_after_lbl,

                "success_strict": strict_ok,
                "flipped_any": any_ok,

                "Baseline_Correct": svm_before_ok,
                "cf_different": cf_different,
                "meaningful_cf": meaningful_cf,
                
                "validated_correct": validated_correct,
                "validated_strict_success": validated_strict_success,

                "distance": dist_abs,
                "rel_distance": dist_rel,

                "y_bin_orig": y_bin_orig,
                "y_bin_cf": y_bin_cf,
                "orig_class_name": CLASS_NAMES.get(svm_before_lbl, f"Class_{svm_before_lbl}"),
                "cf_class_name": CLASS_NAMES.get(svm_after_lbl, f"Class_{svm_after_lbl}"),
            })

    if not results:
        print("No valid results found!")
        return None

    df_all = pd.DataFrame(results)
    df_all["strict_given_baseline"] = df_all["success_strict"] & df_all["Baseline_Correct"]

    # Create summary table with new metrics
    agg = df_all.groupby("Method").agg(
        N=("success_strict", "size"),
        Total_Any_Flips=("flipped_any", "sum"),
        Total_Strict_Flips_from_Correct=("strict_given_baseline", "sum"),
        Total_Validated_Correct=("validated_correct", "sum"),
        Total_Validated_Strict_Success=("validated_strict_success", "sum"),
        Avg_Distance=("distance", "mean"),
        Avg_Rel_Proximity=("rel_distance", "mean"),
    ).reset_index()
    
    # Calculate the three key metrics
    # 1. AFR: Any Flip Rate = Total Any Flips / Total CFs Generated
    agg["AFR"] = agg["Total_Any_Flips"] / agg["N"]
    
    # 2. MSR: Meaningful Success Rate = Strict Flips from Correct / Total CFs Generated
    agg["MSR"] = agg["Total_Strict_Flips_from_Correct"] / agg["N"]
    
    # 3. CSR: Conditional Success Rate
    #    Only counts cases where BOTH ground truth AND SVM agree original was correct
    agg["CSR"] = agg["Total_Validated_Strict_Success"] / agg["Total_Validated_Correct"]
    
    # Reorder columns for better presentation
    agg = agg[["Method", "N", "AFR", "MSR", "CSR", 
               "Total_Validated_Correct",
               "Avg_Distance", "Avg_Rel_Proximity"]]

    pretty = agg.copy()
    pct_cols = ["AFR", "MSR", "CSR"]
    for c in pct_cols:
        pretty[c] = (pretty[c] * 100).round(1).astype(str) + "%"

    pretty["Avg_Distance"] = pretty["Avg_Distance"].round(3)
    pretty["Avg_Rel_Proximity"] = pretty["Avg_Rel_Proximity"].round(3)

    print("\n" + "="*80)
    print("SVM EVALUATION RESULTS:")
    print("="*80)
    print("\nMetric Definitions:")
    print("  AFR (Any Flip Rate): % of CFs that changed prediction to ANY other class")
    print("  MSR (Meaningful Success Rate): % of CFs with correct original -> correct target flip")
    print("  CSR (Conditional Success Rate): Success rate GIVEN ground truth AND SVM agree original was correct")
    print("="*80)
    display(pretty.style.hide(axis='index'))

    result_dict = {
        'results_df': df_all,
        'summary_df': agg,
        'best_svm_model': svm,
    }
    
    if perform_gridsearch:
        result_dict['svm_best_params'] = best_params
        result_dict['svm_best_cv_score'] = best_cv_score

    return result_dict

### Run methods

In [63]:
def run_quick_evaluation(dataset_name, use_ramanspy_preprocessing=False):
    """Quick evaluation with limited hyperparameter search"""
    return run_multiclass_evaluation(
        dataset_name, 
        use_ramanspy_preprocessing,
        perform_gridsearch=True, 
        quick_search=True
    )
def run_comprehensive_evaluation(dataset_name, use_ramanspy_preprocessing=False):
    """Comprehensive evaluation with full hyperparameter search"""
    return run_multiclass_evaluation(
        dataset_name, 
        use_ramanspy_preprocessing,
        perform_gridsearch=True, 
        quick_search=False
    )

In [ ]:
TissueMetrics = run_comprehensive_evaluation(
    "DRS_TissueClassification", 
    use_ramanspy_preprocessing=False)

Running Multi-Class Evaluation for: DRS_TissueClassification
Hyperparameter Tuning: GridSearchCV enabled
Loading reference spectra...
DRS tissue distribution: {np.int64(0): np.int64(1000), np.int64(1): np.int64(1000), np.int64(2): np.int64(1000), np.int64(3): np.int64(1000), np.int64(4): np.int64(1000), np.int64(5): np.int64(215)}
Loaded reference data: X_all.shape=(5215, 1531), y_all.shape=(5215,)
Unique classes in reference: [0 1 2 3 4 5]
After filtering to relevant classes: X_sub.shape=(5215, 1531)
Class distribution: {np.int64(0): np.int64(1000), np.int64(1): np.int64(1000), np.int64(2): np.int64(1000), np.int64(3): np.int64(1000), np.int64(4): np.int64(1000), np.int64(5): np.int64(215)}

--- Training SVM ---
Performing hyperparameter tuning for SVM...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
SVM grid search completed in 225.0 seconds
Best CV score: 0.9871
Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

6-class SVM test accuracy: 0.9895
Confusi

Method,N,AFR,MSR,CSR,Total_Validated_Correct,Avg_Distance,Avg_Rel_Proximity
CELS_global_cels,500,0.4%,0.0%,0.0%,497,1.864000,0.075000
CELS_local_cels,500,1.6%,0.0%,0.0%,497,0.797000,0.032000
Glacier_global_cnn,500,36.4%,23.0%,23.3%,493,1.246000,0.050000
Glacier_global_ne,500,6.0%,3.0%,3.0%,493,1.067000,0.043000
Glacier_local_cnn,500,23.8%,15.8%,16.0%,493,1.392000,0.056000
Glacier_local_ne,500,2.8%,0.8%,0.8%,493,1.020000,0.041000
RSF_global_rsf,500,65.0%,35.2%,35.8%,491,3.436000,0.137000
RSF_local_rsf,500,36.8%,25.2%,25.7%,491,2.061000,0.083000


In [ ]:
CovidMetrics = run_comprehensive_evaluation(
    "RamanCOVID19_ramanspy_preprocessed", 
    use_ramanspy_preprocessing=True)

Running Multi-Class Evaluation for: RamanCOVID19_ramanspy_preprocessed
Preprocessing Mode: ramanspy
Hyperparameter Tuning: GridSearchCV enabled
Loading reference spectra...
Loaded reference data: X_all.shape=(465, 900), y_all.shape=(465,)
Unique classes in reference: [0 1 2]
Applying ramanspy preprocessing to reference data...


Preprocessing completed successfully
After filtering to relevant classes: X_sub.shape=(465, 900)
Class distribution: {np.int64(0): np.int64(150), np.int64(1): np.int64(159), np.int64(2): np.int64(156)}

--- Training SVM ---
Performing hyperparameter tuning for SVM...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
SVM grid search completed in 0.9 seconds
Best CV score: 0.8039
Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'poly'}

3-class SVM test accuracy: 0.8280
Confusion Matrix:
[[26  0  4]
 [ 2 29  1]
 [ 8  1 22]]

Classification Report:
              precision    recall  f1-score   support

     Healthy      0.722     0.867     0.788        30
    COVID-19      0.967     0.906     0.935        32
   Suspected      0.815     0.710     0.759        31

    accuracy                          0.828        93
   macro avg      0.835     0.828     0.827        93
weighted avg      0.837     0.828     0.829        93


Loading and evaluating counterfactuals...
- Evalu

Method,N,AFR,MSR,CSR,Total_Validated_Correct,Avg_Distance,Avg_Rel_Proximity
CELS_global_cels,309,69.3%,2.3%,4.7%,148,0.510000,0.086000
CELS_local_cels,309,80.9%,9.1%,18.9%,148,0.281000,0.048000
Glacier_global_cnn,309,90.3%,8.7%,47.1%,51,2.079000,0.359000
Glacier_global_ne,309,68.0%,2.6%,9.8%,51,1.080000,0.186000
Glacier_local_cnn,309,56.3%,2.6%,13.7%,51,2.299000,0.398000
Glacier_local_ne,309,64.1%,0.6%,2.0%,51,1.025000,0.177000
RSF_global_rsf,309,82.2%,12.0%,26.1%,142,0.567000,0.097000
RSF_local_rsf,309,84.5%,5.8%,12.7%,142,0.647000,0.111000


In [ ]:
BactMetrics = run_comprehensive_evaluation(
    "EcoliVsKpneumoniae_ramanspy_singular", 
    use_ramanspy_preprocessing=True)

Running Multi-Class Evaluation for: EcoliVsKpneumoniae_ramanspy_singular
Preprocessing Mode: ramanspy
Hyperparameter Tuning: GridSearchCV enabled
Loading reference spectra...
Loaded reference data: X_all.shape=(60000, 1000), y_all.shape=(60000,)
Unique classes in reference: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29.]
Applying ramanspy preprocessing to reference data...
Preprocessing completed successfully
After filtering to relevant classes: X_sub.shape=(10000, 1000)
Class distribution: {np.float64(0.0): np.int64(2000), np.float64(1.0): np.int64(2000), np.float64(2.0): np.int64(2000), np.float64(3.0): np.int64(2000), np.float64(9.0): np.int64(2000)}

--- Training SVM ---
Performing hyperparameter tuning for SVM...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
SVM grid search completed in 407.8 seconds
Best CV score: 0.9755
Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

5-class SV

Method,N,AFR,MSR,CSR,Total_Validated_Correct,Avg_Distance,Avg_Rel_Proximity
CELS_global_cels,500,4.2%,2.0%,2.0%,491,1.307000,0.095000
CELS_local_cels,500,8.4%,5.2%,5.3%,491,1.911000,0.139000
Glacier_global_cnn,500,44.2%,34.2%,35.8%,477,1.045000,0.077000
Glacier_global_ne,500,24.6%,16.4%,17.2%,477,0.556000,0.041000
Glacier_local_cnn,500,24.6%,16.4%,17.2%,477,0.939000,0.069000
Glacier_local_ne,500,9.2%,2.2%,2.3%,477,0.699000,0.052000
RSF_global_rsf,500,78.6%,4.4%,5.4%,404,1.380000,0.101000
RSF_local_rsf,500,20.6%,1.6%,2.0%,404,0.974000,0.071000
